# Generator danych do SQL

## Biblioteki

In [1]:
import numpy as np
from math import floor, ceil, log10
import random
import pandas as pd
from datetime import date, timedelta
from faker import Faker

## Zmienne globalne

In [2]:
# ZMIENNE GLOBALNE
DZISIAJ = date.today()

POCZATEK_WYCIECZEK = date(2023, 6, 30)      # Kiedy zaczyna się pierwsza wycieczka
DNI_DO_PRZODU = 60      # Czyli ile dni wprzód planujemy wycieczki (od dnia, w którym generujmey dane)
KONIEC_WYCIECZEK = date.today() + timedelta(days = DNI_DO_PRZODU)       # Kiedy odbywa się ostatnia wycieczka
DNI_WSTECZ = 14     # Maksymalny czas do zapłaty za wycieczkę (czyli płacimy w okresie 60-14 dni do rozpoczęcia)

N_KLIENCI = 10000       # Można zmieniać, jednak obecna liczba jest optymalna (średnio przypada ok. 1-1,5 wycieczki na osobę)
N_PRACOWNICY = 15       # Stała wartość
N_ALARMOWE = floor(0.85*N_KLIENCI)      # <= N_KLIENCI, neiktórzy mogą podać innych klientów jako osoba do kontaktu w razie wypadku

OSOB_NA_START = 500               # Tyle pierwszych osób w tej firmie od pierwszej wycieczki
MAX_TRWAJACYCH_WYCIECZEK = 2        # Jednocześnie może trwać tyle wycieczek naraz
MINIMUM_NA_WYCIECZKE = 30       # Tyle min osób na wycieczke
MAKSIMUM_NA_WYCIECZKE = 50      # Tyle max osób na wycieczke

fake = Faker()

data_nan = date(1, 1, 1)
liczba_nan = 0
napis_nan = "NULL"

## Generowanie imion i nazwisk klientów, pracowników i alarmowych.
Do tego generuje stanowiska i przypisuje je pracownikom.

In [3]:
n = N_KLIENCI + N_PRACOWNICY + N_ALARMOWE
m = 0

# IMIONA
print("Generuję imiona")

file_path = '../Dane do generatora/imiona.csv'
df = pd.read_csv(file_path)
liczba_wierszy = df.shape[0]
kolumna_wartosci = df.columns[1]
df['Dystrybuanta'] = df[kolumna_wartosci].cumsum()
dystrybuanty = df['Dystrybuanta']

maks = dystrybuanty[liczba_wierszy - 1]

losowe = np.random.randint(maks, size = n)

wyniki1 = []
for los in losowe:
    a = 0
    b = liczba_wierszy - 1
    while True:
        i = (a + b)//2
        if abs(a-b) <= 1:
            break
        if los < dystrybuanty[i]:
            b = i
            continue
        if los > dystrybuanty[i]:
            a = i
            continue
        break
    plec = df.iloc[i, 2]
    imie = df.iloc[i, 0]
    if plec == 'M':
        m += 1
        wyniki1.insert(0, imie)
        continue
    wyniki1.append(imie)

# NAZWISKA MĘSKIE
print("Generuję nazwiska dla mężczyzn")

file_path = '../Dane do generatora/nazwiska_męskie.csv'
df = pd.read_csv(file_path)
liczba_wierszy = df.shape[0]
kolumna_wartosci = df.columns[1]
df['Dystrybuanta'] = df[kolumna_wartosci].cumsum()

dystrybuanty = df['Dystrybuanta']

maks = dystrybuanty[liczba_wierszy - 1]
losowe = np.random.randint(maks, size = m)

wyniki2 = []
for los in losowe:
    a = 0
    b = liczba_wierszy - 1
    while True:
        i = (a + b)//2
        if abs(a-b) <= 1:
            break
        if los < dystrybuanty[i]:
            b = i
            continue
        if los > dystrybuanty[i]:
            a = i
            continue
        break
    nazwisko = df.loc[i, 'Nazwisko aktualne']
    wyniki2.append(nazwisko)

#NAZWISKA DAMSKIE
print("Generuję nazwiska dla kobiet")
file_path = '../Dane do generatora/nazwiska_damskie.csv'
df = pd.read_csv(file_path)
liczba_wierszy = df.shape[0]
kolumna_wartosci = df.columns[1]
df['Dystrybuanta'] = df[kolumna_wartosci].cumsum()

dystrybuanty = df['Dystrybuanta']

maks = dystrybuanty[liczba_wierszy - 1]
losowe = np.random.randint(maks, size = n - m)

for los in losowe:
    a = 0
    b = liczba_wierszy - 1
    while True:
        i = (a + b)//2
        if abs(a-b) <= 1:
            break
        if los < dystrybuanty[i]:
            b = i
            continue
        if los > dystrybuanty[i]:
            a = i
            continue
        break
    nazwisko = df.loc[i, 'Nazwisko aktualne']
    wyniki2.append(nazwisko)

#NUMERY TELEFONÓW
print("Generuję numery telefonów")

telefony = set()
while len(telefony)<n:
    numer = str(np.random.randint(4, 10))
    for j in range(8):
        numer = numer + str(np.random.randint(10))
    telefony.add(numer)

wyniki5 = list(telefony)

print("Sklejam imiona i nazwiska")
wyniki = [(wyniki1[i], wyniki2[i]) for i in range(n)]

random.shuffle(wyniki)

# NAJPIERW MAMY PRACOWNIKÓW,
# POTEM KLIENTÓW I NUMERY ALARMOWE
# DZIELIMY ICH I PRZYPISUJEMY DANE

print("Dzielę tabelę na klientów, pracowników i numery alarmowe")
def podział_listy(lista, ile):
    return lista[:ile], lista[ile:]

pracownicy_imie_nazwisko, reszta_imie_nazwisko = podział_listy(wyniki, N_PRACOWNICY)
pracownicy_telefon, reszta_telefon = podział_listy(wyniki5, N_PRACOWNICY)

pracownicy = {"id_pracownika": [x+1 for x in range(N_PRACOWNICY)],
              "imie": [pracownicy_imie_nazwisko[i][0] for i in range(N_PRACOWNICY)],
              "nazwisko": [pracownicy_imie_nazwisko[i][1] for i in range(N_PRACOWNICY)],
              "numer_telefonu": pracownicy_telefon}

klienci_imie_nazwisko, alarm_imie_nazwisko = podział_listy(reszta_imie_nazwisko, N_KLIENCI)
klienci_telefon, alarm_telefon = podział_listy(reszta_telefon, N_KLIENCI)

osoby = {"id_osoby": [x+1 for x in range(N_KLIENCI)],
         "imie": [klienci_imie_nazwisko[i][0] for i in range(N_KLIENCI)],
         "nazwisko": [klienci_imie_nazwisko[i][1] for i in range(N_KLIENCI)],
         "numer_telefonu": klienci_telefon}

dane_alarmowe = {"imie": [alarm_imie_nazwisko[i][0] for i in range(N_ALARMOWE)],
                 "nazwisko": [alarm_imie_nazwisko[i][1] for i in range(N_ALARMOWE)],
                 "numer_telefonu": alarm_telefon}

# E-MAILE
print("Generuję e-maile")

# GENERATOR UNIKATOWYCH ZBITEK NUMERÓW
domeny = ["@o2.pl","@gmail.com","@wp.pl"]
numery = [str(x+1) for x in range(N_KLIENCI)]
random.shuffle(numery)

# E-MAILE KLIENTÓW
emaile_klientów = []
błędy = []
for kto in range(N_KLIENCI):
    im = osoby["imie"][kto][0]
    naz = osoby["nazwisko"][kto]
    emaile_klientów.append((im + naz + numery[kto] + random.choice(domeny)).lower())

osoby["email"] = emaile_klientów

# E-MAILE PRACOWNIKÓW
emaile_prac = []
for kto in range(N_PRACOWNICY):
    im = pracownicy["imie"][kto]
    naz = pracownicy["nazwisko"][kto]
    num = str(pracownicy["id_pracownika"][kto])

    emaile_prac.append((im + "." + naz + num + "@wombat.pl").lower())

pracownicy["e-mail"] = emaile_prac

# STANOWISKA
print("Generuję stanowiska")

file_path = '../Dane do generatora/stanowiska.xlsx'
stanowiska = pd.read_excel(file_path, header=0)

jakie_stanowiska = ["Koordynator wycieczki"] * 4 + ["Fotograf"] * 4 + ["Organizator wycieczki"] * 2 + ["Sprzątaczka"] + ["Sekretarka"] + ["Księgowa"] + ["Ochroniarz"] + ["Specjalista ds. Marketingu"]
random.shuffle(jakie_stanowiska)

id_stanowisk_pracowników = []
for nazwa in jakie_stanowiska:
    id_stanowiska = stanowiska[stanowiska['stanowisko']==nazwa]['id_stanowiska'].item()
    id_stanowisk_pracowników.append(id_stanowiska)

pracownicy["id_stanowiska"] = id_stanowisk_pracowników
pracownicy["czy_zatrudniony"] = [1 for x in range(N_PRACOWNICY)]

# DATA PODPISANIA UMOWY
pracownicy["data_zatrudnienia"] = [fake.date_between(start_date=POCZATEK_WYCIECZEK-timedelta(days=28), end_date=POCZATEK_WYCIECZEK) for x in range(N_PRACOWNICY)]

print("Koniec")
pracownicy = pd.DataFrame(pracownicy)
osoby = pd.DataFrame(osoby)

Generuję imiona
Generuję nazwiska dla mężczyzn
Generuję nazwiska dla kobiet
Generuję numery telefonów
Sklejam imiona i nazwiska
Dzielę tabelę na klientów, pracowników i numery alarmowe
Generuję e-maile
Generuję stanowiska
Koniec


In [4]:
#Do id_alarmowe dodajemy ok. 15% klientów

ile_klientów = N_KLIENCI - N_ALARMOWE
wiersze_osoby = [i for i in range(osoby.shape[0])]
random.shuffle(wiersze_osoby)
wiersze_osoby = wiersze_osoby[:ile_klientów]

for i in wiersze_osoby:
    dane_alarmowe["imie"].append(osoby.loc[i, "imie"])
    dane_alarmowe["nazwisko"].append(osoby.loc[i, "nazwisko"])
    dane_alarmowe["numer_telefonu"].append(osoby.loc[i, "numer_telefonu"])

dane_alarmowe["id_alarm_pomocnicze"] = [i+1 for i in range(len(dane_alarmowe["imie"]))]

In [5]:
id_alarm = {}
for i in dane_alarmowe["id_alarm_pomocnicze"]:
    id_alarm[i] = 0

maksimum_alarm = 3

id_alarm_osoby = []

while len(id_alarm_osoby) < osoby.shape[0]:
    dodanie = False
    while not dodanie:
        idd = random.choice(dane_alarmowe["id_alarm_pomocnicze"])
        if id_alarm[idd] < maksimum_alarm:
            id_alarm_osoby.append(idd)
            id_alarm[idd] += 1
            dodanie = True

#Usuń jeszcze wiersze
do_usunięcia = []

for i in id_alarm.keys():
    if id_alarm[i] == 0: do_usunięcia.append(i)

for i in do_usunięcia: del id_alarm[i]

In [6]:
for i in range(len(do_usunięcia)):
    do_usunięcia[i] -= 1
    #Usuwać w DataFrame robimy po indeksach, a nie id_alarmowych, więc musimy zmniejszyć o 1


dane_alarmowe = pd.DataFrame(dane_alarmowe)
dane_alarmowe.drop(do_usunięcia, inplace=True)
dane_alarmowe.reset_index(drop=True, inplace=True)

In [7]:
x = []
for i in id_alarm_osoby:
    if i not in x: x.append(i)


dane_alarmowe["id_alarm_pomocnicze"] = pd.Categorical(dane_alarmowe["id_alarm_pomocnicze"], categories=x, ordered=True)

dane_alarmowe.sort_values("id_alarm_pomocnicze", inplace=True)

dane_alarmowe.reset_index(drop=True, inplace=True)

In [8]:
id_alarm = []
for i in range(dane_alarmowe.shape[0]):
    id_alarm.append(i+1)

dane_alarmowe["id_alarm"] = id_alarm


In [9]:
for i in range(len(id_alarm_osoby)):
    id_alarm_osoby[i] = dane_alarmowe[dane_alarmowe["id_alarm_pomocnicze"]==id_alarm_osoby[i]]["id_alarm"].item()

osoby["id_alarm"] = id_alarm_osoby

## Generowanie miast, kierunków, rodzajow i ofert i cennika

In [10]:
df_wszystkie = [
    ("Survival z węgierskim poradnikiem przetrwania", ["las", "góry"], "survival"),
    ("Babciowy escape room, czyli ucieczka przed kolejną dokładką", ["kamienica", "stara wiejska chata"], "escape room"),
    ("Poszukiwanie Świętego Górala", ["góry", "las", "ruiny zamku", "twierdza"], "humor"),
    ("Niespodziewana rekonstrukcja hiszpańskiej inkwizycji", ["katakumby", "skansen", "twierdza"], "humor"),
    ("Skok za spadochronem", ["lotnisko"], "ekstremalne"),
    ("Konkurs na najlepszy tuning Polonez", ["parking", "tor wyścigowy"], "motoryzacja"),
    ("Wyścigi 'Maluchów' z silnikami V8", ["tor wyścigowy"], "motoryzacja"),
    ("Misja szpiegowska: infiltracja opuszczonej kamienicy w stroju Lajkonika", ["kamienica", "rynek miejski"], "fabularne"),
    ("Misja szpiegowska: Tajna misja w pełnej zbroi płytowej", ["twierdza", "ruiny zamku", "skansen", "katakumby"], "fabularne"),
    ("Konkurs kulinarny: Najdziwniejszy kebab", ["rynek miejski", "stare miasto"], "kulinaria"),
    ("Konkurs kulinarny: 101 dań z cebuli", ["rynek miejski", "stare miasto"], "kulinaria"),
    ("Konkurs kulinarny: Niedorzeczne połączenia", ["rynek miejski", "stare miasto"], "kulinaria"),
    ("Zwiedzanie ogrodu botanicznego w masce przeciwgazowej", ["ogród botaniczny"], "rekreacja"),
    ("Skok na grzybobranie (dosłownie)", ["lotnisko"], "ekstremalne"),
    ("Nocne grzybobranie w noktowizorach", ["las"], "grzybobranie"),
    ("Grzybobranie na poligonie wojskowym", ["poligon wojskowy"], "grzybobranie"),
    ("Grzybobranie podczas inscenizacji polowania", ["las"], "grzybobranie"),
    ("Golf na szczudłach", ["park miejski", "pole golfowe"], "sport"),
    ("Karaoke na helu", ["rynek miejski", "park miejski", "klub muzyczny", "dom kultury"], "muzyczne"),
    ("Wyścig w wózkach sklepowych", ["parking", "tor wyścigowy", "stadion sportowy"], "wyścigi"),
    ("Metalowy koncert na lirze korbowej i burczybasie", ["rynek miejski", "dom kultury", "klub muzyczny"], "muzyczne"),
    ("Wycieczka rowerowa z Ojcem Matateusz", ["rynek miejski", "park miejski", "stare miasto"], "turystyka"),
    ("Pomysł na najbardziej niedorzeczne ministerstwo", ["rynek miejski"], "kreatywność"),
    ("Kurs normalizacji baz danych w języku chińskim", ["opuszczona szkoła", "zakład psychiatryczny"], "humor"),
    ("Noc RPGów, ale to liczby zespolone", ["park miejski", "dom kultury", "rynek miejski"], "planszowe"),
    ("Grzybobranie na szczudłach", ["las"], "grzybobranie"),
    ("Konkurs na najbardziej zbędny gadżet", ["park miejski", "rynek miejski"], "kreatywność"),
    ("Konkurs jedzenia kiszonego śledzia", ["rynek miejski", "park miejski"], "kulinaria"),
    ("Najlepsze przebranie za znak drogowy", ["rynek miejski"], "kostiumy"),
    ("Półmaraton w strojach T-Rexa", ["stadion sportowy", "park miejski"], "sport"),
    ("Noc gier planszowych ze zmianą miejsc", ["dom kultury", "rynek miejski", "park miejski"], "planszowe"),
    ("Szachy warcabami", ["dom kultury", "rynek miejski"], "planszowe"),
    ("Gra Łasicy", ["kamienica", "rynek miejski", "twierdza"], "ekstremalne"),
    ("Walka na szable w okularach zniekształcających rzeczywistość", ["skansen", "ruiny zamku", "twierdza"], "humor"),
    ("Ucieczka z więzienia w stroju klauna", ["zakład karny", "zakład psychiatryczny"], "escape room")
]


miasta = ["Warszawa", "Sosnowiec","Gdańsk", "Nowigrad", "Łódź", "Kielce", "Białystok", "Zielona Góra", "Bydgoszcz", "Jęczydół", "Kiełbasy", "Rokitki", "Kłopotów", "Rzeszotary", "Prusy", "Owsianka", "Ciepłowody", "Uraz", "Wrocław", "Sandomierz", "Wadowice", "Radom"]
id_miasta = [id for id in range(1,len(miasta)+1)]
#for i in range(len(miasta)):
#    for miasto in miasta:
#        df_miasta = {}

lokacje = list({lok for oferta in df_wszystkie for lok in oferta[1]})
id_kier= {lok: id+1 for id, lok in enumerate(lokacje)  }

rodzaje = list({oferta[2] for oferta in df_wszystkie })
id_rodzaje = {rodzaj: id + 1 for id,rodzaj  in enumerate(rodzaje)}

df_oferty = [(elem[0], id_kier[random.choice(elem[1])], id_rodzaje[elem[2]] ) for elem in df_wszystkie]
df_oferty = pd.DataFrame(df_oferty, columns= ["nazwa", "id_kierunku", "id_rodzaju"])
df_oferty["id_miasta"] = np.random.randint(1,len(miasta), size=df_oferty.shape[0])

df_kier = [(id + 1, lok) for id, lok in enumerate(lokacje)]
df_kier = pd.DataFrame(df_kier, columns= ["id_kierunku", "miejsce"])

df_rodzaje = [(id + 1, rodzaj) for id, rodzaj in enumerate(rodzaje)] 
df_rodzaje = pd.DataFrame(df_rodzaje, columns= ["id_rodzaju", "rodzaj"])

df_miasta = pd.DataFrame(data = [id_miasta,miasta])
df_miasta.values.tolist()

df_miasta_temp = {'id_miasta': [],
                 'miasto': []}

df_miasta_temp['id_miasta'] = id_miasta
df_miasta_temp['miasto'] = miasta

df_miasta = pd.DataFrame(df_miasta_temp)

cennik = pd.read_csv("../Dane do generatora/cennik.csv", sep = ";")
stanowiska = pd.read_excel("../Dane do generatora/stanowiska.xlsx")

In [11]:
#Wczytywanie opisów
opisy = pd.read_excel('../Dane do generatora/wycieczki.xlsx', header=0)
opisy['id'] = 0
opisy['id_cennika'] = 0
for i in range(opisy.shape[0]):
    indeks = i+1
    opisy.loc[i, 'id'] = indeks
    opisy.loc[i, 'id_cennika'] = indeks

df_oferty['opis'] = opisy['opis']
df_oferty['czas_trwania'] = opisy['czas_trwania (dni)']
df_oferty['czy_bezpieczna'] = opisy['czy_bezpieczna']
df_oferty['id'] = opisy['id']

oferty = df_oferty[['id', 'nazwa', 'opis', 'id_rodzaju', 'id_kierunku', 'id_miasta', 'czas_trwania', 'czy_bezpieczna']]
LICZBA_PLANOW = oferty.shape[0]

In [12]:
def naprawa_bledow_jaka_wycieczka(jaka_wycieczka):
    ilość_dat = {}
    jaka_data = POCZATEK_WYCIECZEK
    while jaka_data <= KONIEC_WYCIECZEK:
        ilość_dat[jaka_data] = 0  
        jaka_data += timedelta(days=1)
    
    błędy = []
    
    #Sprawdzamy ilość błędów typu "tego samego dnia wycieczki z tymi samymi planami"
    for plan in oferty['id']:
        daty = jaka_wycieczka[jaka_wycieczka['id_planu']==plan]['data']
        for i in range(len(daty)-1):
            for j in range(i+1, len(daty)):
                if daty.iloc[i] == daty.iloc[j]: błędy.append(f"BŁĄD ten sam dzień: {plan}")


    #Sprawdzamy ilość błędów typu "wycieczka o tym planie już się odbywa"
    for indeks in range(jaka_wycieczka.shape[0]):
        id = jaka_wycieczka.loc[indeks, 'id_planu']
        długość = oferty[oferty['id'] == id]['czas_trwania']
        długość = długość.item()
        for dni in range(długość-1):
            data = jaka_wycieczka.loc[indeks, 'data'] - timedelta(days=dni+1)
            wycieczki_o_dacie = jaka_wycieczka[jaka_wycieczka['data']==data]['id_planu']

            if id in jaka_wycieczka[jaka_wycieczka['data']==data]['id_planu'].values: błędy.append(f"BŁĄD jakaś już trwa: {indeks}")
    #print(len(błędy))
    
    #Naprawiamy poprzednie błędy

    błędy_both = True

    while błędy_both:

        błędy_both = False
        błędy1 = True
        błędy2 = True

        while błędy1:

            błędy1 = False
            indeksy = []
            for plan in oferty['id']:

                wycieczka_plan = jaka_wycieczka[jaka_wycieczka['id_planu']==plan]
                daty = wycieczka_plan['data']
                ilość = len(wycieczka_plan)

                for i in range(ilość-1):
                    for j in range(i+1, ilość):

                        if daty.iloc[i] == daty.iloc[j]:
                            błędy1 = True
                            indeksy.append(wycieczka_plan.iloc[i]['id'])

            for k in indeksy:

                wygenerowana_data = fake.date_between(start_date=POCZATEK_WYCIECZEK, end_date=KONIEC_WYCIECZEK)
                zła_data = True

                while zła_data:

                    if ilość_dat[wygenerowana_data] < jaka_wycieczka.shape[0]:

                        ilość_dat[jaka_wycieczka.loc[k-1, 'data']] -= 1
                        ilość_dat[wygenerowana_data] += 1
                        jaka_wycieczka.loc[k, 'data'] = wygenerowana_data

                        zła_data = False

                    else: wygenerowana_data = fake.date_between(start_date=POCZATEK_WYCIECZEK, end_date=KONIEC_WYCIECZEK)


        while błędy2:

            błędy2 = False

            for indeks in range(jaka_wycieczka.shape[0]):

                id = jaka_wycieczka.loc[indeks, 'id_planu']
                długość = oferty[oferty['id'] == id]['czas_trwania']
                długość = długość.item()

                for dni in range(1, długość):
                    data = jaka_wycieczka.loc[indeks, 'data'] - timedelta(days=dni)
                    wycieczki_o_dacie = jaka_wycieczka[jaka_wycieczka['data']==data]['id_planu']

                    if id in wycieczki_o_dacie.values:

                        jaka_wycieczka.loc[indeks, 'id_planu'] = random.randint(1, LICZBA_PLANOW)
                        błędy2 = True

        if błędy1 == True or błędy2 == True: błędy_both = True
    
    poprawione_błędy = []
    #Sprawdzamy, czy poprawione
    for plan in oferty['id']:
        daty = jaka_wycieczka[jaka_wycieczka['id_planu']==plan]['data']
        for i in range(len(daty)-1):
            for j in range(i+1, len(daty)):
                if daty.iloc[i] == daty.iloc[j]: poprawione_błędy.append(f"BŁĄD ten sam dzień: {plan}")

    #Sprawdzamy ilość błędw typu "wycieczka o tym planie już się odbywa"
    for indeks in range(jaka_wycieczka.shape[0]):
        id = jaka_wycieczka.loc[indeks, 'id_planu']
        długość = oferty[oferty['id'] == id]['czas_trwania']
        długość = długość.item()
        for dni in range(długość-1):
            data = jaka_wycieczka.loc[indeks, 'data'] - timedelta(days=dni+1)
            wycieczki_o_dacie = jaka_wycieczka[jaka_wycieczka['data']==data]['id_planu']

            if id in jaka_wycieczka[jaka_wycieczka['data']==data]['id_planu'].values: poprawione_błędy.append(f"BŁĄD ten sam dzień: {indeks}")
    #for i in błędy: #print(i)
    #print(f"Czy udało się naprawić błędy? {poprawione_błędy==[]}")
    #if poprawione_błędy!=[]: print("\nNIE DZIAŁA\n", poprawione_błędy)
    
    #Naprawiamy kolejność + indeksy w jakie_wycieczki
    jaka_wycieczka = jaka_wycieczka.sort_values(by='data')
    jaka_wycieczka['id'] = [x+1 for x in range(jaka_wycieczka.shape[0])]
    #print(jaka_wycieczka)

In [13]:
jaka_wycieczka = {'id_planu': [],
                 'liczba_osób': [],
                 'data': [],
                 'id': []}
id_jaka_wycieczka = 1

obecna_data = POCZATEK_WYCIECZEK


trwajace_wycieczki = []
for i in range(MAX_TRWAJACYCH_WYCIECZEK):
    trwajace_wycieczki.append(POCZATEK_WYCIECZEK)

while obecna_data < KONIEC_WYCIECZEK:
    obecna_data += timedelta(days = random.choice([0, 1]))
    for i in range(MAX_TRWAJACYCH_WYCIECZEK):
        if obecna_data > trwajace_wycieczki[i]:
            if random.randint(1, 10) < 4:
                plan = random.randint(1, LICZBA_PLANOW)
                czas_trwania = int(oferty['czas_trwania'][plan - 1])
                trwajace_wycieczki[i] = obecna_data + timedelta(days = czas_trwania - 1)
                
                jaka_wycieczka['id_planu'].append(plan)
                jaka_wycieczka['liczba_osób'].append(random.randint(MINIMUM_NA_WYCIECZKE, MAKSIMUM_NA_WYCIECZKE))
                jaka_wycieczka['data'].append(obecna_data)
                jaka_wycieczka['id'].append(id_jaka_wycieczka)
                
                id_jaka_wycieczka += 1
            break
            
jaka_wycieczka = pd.DataFrame(jaka_wycieczka)
naprawa_bledow_jaka_wycieczka(jaka_wycieczka)

In [14]:
# Przygotowuję każdą osobę do wyboru
kazda_osoba = osoby
liczba_osob = kazda_osoba.shape[0]
liczba_wycieczek = jaka_wycieczka.shape[0]
wspolczynnik_przyrostu = round((liczba_osob - OSOB_NA_START)/(liczba_wycieczek**2), 3)

kazda_osoba['zajeta'] = [False for _ in range(liczba_osob)]

start_date = jaka_wycieczka['data'][0] # Data pierwszej wycieczki
kazda_osoba['min_data'] = [start_date for _ in range(liczba_osob)]

kazda_osoba = kazda_osoba[['id_osoby', 'zajeta', 'min_data']]
kazda_osoba = kazda_osoba.values.tolist()

# Przyszła tabela osoba_wycieczka
osoba_wycieczka = {'id_wycieczki': [],
                   'id_osoby': [],
                   'czy_zapłacono': [],
                   'data_zapłaty': [],
                   'ocena': []}


# Przygotowuję każdą wycieczkę do przelatywania
wycieczki = jaka_wycieczka
dlugosci = []

for id_planu in jaka_wycieczka['id_planu']:
    id_planu = oferty.loc[id_planu - 1]['czas_trwania']
    dlugosci.append(id_planu)
    
wycieczki['długość'] = dlugosci

wycieczki = wycieczki[['id', 'liczba_osób', 'data', 'długość']]
wycieczki = wycieczki.values.tolist()

# ---------------------
# Oceny
#
#   Ustalamy podzielnik procentowy
podzial_proc = 15
#   Każda z poniższych zmiennych powinna zawierać liczby <= podzial_proc
szansa_wystawienia_oceny = 12
szanse_ocen = [1, 3, 5, 8, 15]
liczba_ocen = 0
#
# ---------------------

for wycieczka in wycieczki:
    obecna_data = wycieczka[2]
    nastepna_data_min = obecna_data + timedelta(days = wycieczka[3])
    
    # Dopóki liczba osób wycieczki jest dodatnia
    while wycieczka[1] > 0:
        wylosowana_osoba = random.randint(1, min(liczba_osob, floor(OSOB_NA_START + wspolczynnik_przyrostu * wycieczka[0]**2))) # Losuję osobę
        if kazda_osoba[wylosowana_osoba - 1][1]: # Jeśli jest zajęta, to ponawiam losowanie
            continue
        # Dodaję osobę, która pojedzie na tę wycieczkę. Zapewniam, że na czas wycieczki jest ona zajęta.
        kazda_osoba[wylosowana_osoba - 1][1] = True
        kazda_osoba[wylosowana_osoba - 1][2] = nastepna_data_min
        osoba_wycieczka['id_osoby'].append(wylosowana_osoba)
        osoba_wycieczka['id_wycieczki'].append(wycieczka[0])
        # Generuję dla osoby datę wpłaty
        if obecna_data <= DZISIAJ + timedelta(days = DNI_WSTECZ):
            płatność = fake.date_between(start_date = obecna_data - timedelta(DNI_DO_PRZODU), end_date=min(DZISIAJ, obecna_data - timedelta(days = DNI_WSTECZ)))
            osoba_wycieczka["czy_zapłacono"].append(1)
            osoba_wycieczka["data_zapłaty"].append(płatność)
        else:
            zapłata = int(random.choice([0, 1]))
            osoba_wycieczka["czy_zapłacono"].append(zapłata)
            if zapłata == 0:
                osoba_wycieczka["data_zapłaty"].append(data_nan)
            else:
                osoba_wycieczka["data_zapłaty"].append(fake.date_between(start_date = obecna_data - timedelta(DNI_DO_PRZODU), end_date = DZISIAJ))
        if nastepna_data_min < DZISIAJ:
            if random.randint(1, podzial_proc) < szansa_wystawienia_oceny:
                los = random.randint(1, podzial_proc)
                ocena : int = 1
                for szansa_ocen in szanse_ocen:
                    if los <= szansa_ocen:
                        osoba_wycieczka['ocena'].append(ocena)
                        break
                    ocena += 1
            else:
                osoba_wycieczka['ocena'].append(liczba_nan)
            liczba_ocen += 1
        else:
            osoba_wycieczka['ocena'].append(liczba_nan)
            
        # Po dodaniu osoby do wycieczki odejmuję jej wymaganą liczbę osób o 1
        wycieczka[1] -= 1
    # Aktualizuję osoby, które wróciły już z wycieczki
    for osoba in kazda_osoba:
        if osoba[2] >= obecna_data:
            osoba[1] = False

#print("Wygenerowano pomyślnie", jaka_wycieczka['liczba_osób'].sum(), "rekordów.")

osoba_wycieczka = pd.DataFrame(osoba_wycieczka)

In [15]:
osoba_wycieczka[osoba_wycieczka['data_zapłaty'] >= DZISIAJ].head(60)

,id_wycieczki,id_osoby,czy_zapłacono,data_zapłaty,ocena


In [16]:
osoba_wycieczka.head(5)

,id_wycieczki,id_osoby,czy_zapłacono,data_zapłaty,ocena
0,1,173,1,2023-06-03,2
1,1,453,1,2023-05-21,0
2,1,436,1,2023-06-03,2
3,1,128,1,2023-05-18,4
4,1,54,1,2023-05-21,5


In [17]:
#Sprawdzamy, czy na pewno na daną wycieczkę wpisane mamy tyle osób, ile powinno jechać

for i in range(jaka_wycieczka.shape[0]):
    if osoba_wycieczka[osoba_wycieczka['id_osoby']==i+1].shape[0] == jaka_wycieczka.loc[i, 'liczba_osób']:
        print("Powinno być", jaka_wycieczka.loc[i, 'liczba_osób'], "a jest", osoba_wycieczka[osoba_wycieczka['id_osoby']==i+1].shape[0])
        print(osoba_wycieczka[osoba_wycieczka['id_wycieczki'] == i])

#Nic nie printuje, więc jest poprawnie

In [18]:
y = 0
for i in jaka_wycieczka['liczba_osób']:
    y += i
print(y==osoba_wycieczka.shape[0]) #True, czyli mamy tyle indeksów, ile jest wycieczek

True


In [19]:
minimum = min(osoba_wycieczka[(osoba_wycieczka['data_zapłaty'])==date(1, 1, 1)]['id_wycieczki'])

print("Pierwsza wycieczka, dla której nie ma obowiązku zapłaty ma id:", minimum)
print(jaka_wycieczka.iloc[minimum-1]['data'] > DZISIAJ + timedelta(days=DNI_WSTECZ))
print(jaka_wycieczka.iloc[minimum-2]['data'] <= DZISIAJ + timedelta(days=DNI_WSTECZ))

#Poprawne daty płatności

Pierwsza wycieczka, dla której nie ma obowiązku zapłaty ma id: 315
True
True


## Dobieranie koordynatorów i fotografów do wycieczek (z uwzględnieniem urlopów)

In [20]:
#Ze względu na losowość generowanych stanowisk, musimy sprawdzić id_pracowników o odpowiednim id_stanowiska

id_koordynator = stanowiska[stanowiska['stanowisko']=="Koordynator wycieczki"]['id_stanowiska'].item()
id_fotograf = stanowiska[stanowiska['stanowisko']=="Fotograf"]['id_stanowiska'].item()

koordynatorzy = []
fotografowie = []

for i in pracownicy["id_pracownika"]:
    if pracownicy.loc[i-1, "id_stanowiska"] == id_koordynator: koordynatorzy.append(i)
    elif pracownicy.loc[i-1, "id_stanowiska"]  == id_fotograf: fotografowie.append(i)

In [21]:
#Generujemy urlopy

def generator_urlopow(N):
    N_PRACOWNICY = N  # Liczba pracowników
    dni = 20
    
    urlopy = []
    for rok in range(2024, 2025+1):
        POCZATEK_URLOPOW = date(rok, 2, 1)
        KONIEC_URLOPOW = date(rok, 12, 31)

        zajete_dni_w_roku = set()  # Zbiór zajętych dni dla wszystkich pracowników w danym roku

        for pracownik in range(1, N_PRACOWNICY + 1):
            while True:
                start_urlopu = fake.date_between(start_date=POCZATEK_URLOPOW, end_date=KONIEC_URLOPOW - timedelta(days=dni))
                dni_urlopu = [start_urlopu + timedelta(days=i) for i in range(dni)]

                # Sprawdzamy, czy urlopy nie pokrywają się
                if not zajete_dni_w_roku.intersection(dni_urlopu):
                    zajete_dni_w_roku.update(dni_urlopu)
                    koniec_urlopu = start_urlopu + timedelta(days=dni-1)
                    urlopy.append((start_urlopu , koniec_urlopu))
                    break
    return urlopy

daty_urlopu_koordynatorów = generator_urlopow(4)
daty_urlopu_fotografów = generator_urlopow(4)

In [22]:
#Generujemy urlopy oddzielnie dla koordynatorów oraz fotografów, gdyż w poszczególnej grupie jednocześnie urlop może maksymalnie 1 osoba

urlopy_koordynatorzy = {}
urlopy_fotografowie = {}

random.shuffle(koordynatorzy)
random.shuffle(fotografowie)

data = 0
for pracownik in koordynatorzy:
    urlopy_koordynatorzy[pracownik] = (daty_urlopu_koordynatorów[data], daty_urlopu_koordynatorów[data+4])
    data +=1

data=0
for pracownik in fotografowie:
    urlopy_fotografowie[pracownik] = (daty_urlopu_fotografów[data], daty_urlopu_fotografów[data+4])
    data +=1

In [23]:
#Tworzymy oddzielne ramki danych dla koordynatorów i fotografów
#Zgodnie ze schematem pojawiają się id_wycieczki oraz id_pracownika, natomiast ile_trwa i data_rozpoczecia_wycieczki są kolumnami pomocniczymi

id_w = []
for i in jaka_wycieczka["id"]:
    id_w.append(i)

trwanie = []
for i in jaka_wycieczka["id_planu"]:
    trwanie.append(int(opisy.loc[i-1, "czas_trwania (dni)"]))

rozpoczęcie = []
for i in range(jaka_wycieczka.shape[0]):
    rozpoczęcie.append(jaka_wycieczka.loc[i, "data"])

koordynatorzy_wycieczka = {"id_wycieczki": id_w, "id_pracownika": [0 for x in id_w], "ile_trwa": trwanie, "data_rozpoczecia_wycieczki": rozpoczęcie}
koordynatorzy_wycieczka = pd.DataFrame(koordynatorzy_wycieczka)

fotografowie_wycieczka = {"id_wycieczki": id_w, "id_pracownika": [0 for x in id_w], "ile_trwa": trwanie, "data_rozpoczecia_wycieczki": rozpoczęcie}
fotografowie_wycieczka = pd.DataFrame(fotografowie_wycieczka)

In [24]:
#Dobieramy pierwszych pracowników do wycieczek

kiedy_jedzie_koordynator = {}

for i in range(len(koordynatorzy)):
    id = koordynatorzy[i]
    od = koordynatorzy_wycieczka.loc[i, "data_rozpoczecia_wycieczki"]
    do = od + timedelta(days=int(koordynatorzy_wycieczka.loc[i, "ile_trwa"]-1))
    
    koordynatorzy_wycieczka.loc[i, "id_pracownika"] = koordynatorzy[i]
    kiedy_jedzie_koordynator[id] = (od, do)


kiedy_jedzie_fotograf = {}

for i in range(len(fotografowie)):
    id = fotografowie[i]
    od = fotografowie_wycieczka.loc[i, "data_rozpoczecia_wycieczki"]
    do = od + timedelta(days=int(fotografowie_wycieczka.loc[i, "ile_trwa"]-1))
    
    fotografowie_wycieczka.loc[i, "id_pracownika"] = fotografowie[i]
    kiedy_jedzie_fotograf[id] = (od, do)

In [25]:
def naprawa_bledow_pracownik_wycieczka(wycieczka, pracownicy, urlopy, kiedy_jedzie):
    
    i = 4
    przypisani = 0
    
    while i < wycieczka.shape[0]:
    
        if przypisani == 4:
            random.shuffle(pracownicy)
            przypisani = 0

        początek = wycieczka.loc[i, "data_rozpoczecia_wycieczki"]
        ile = int(wycieczka.loc[i, "ile_trwa"])
        koniec = początek + timedelta(days=ile-1)
        #print(początek, ile, koniec)
    
        
        dodawanie = True
        
        kto = pracownicy[przypisani]
        przypisani += 1
        #print(kto, przypisani)
        
        początek_urlopu_2024 = urlopy[kto][0][0]
        koniec_urlopu_2024 = urlopy[kto][0][1]
        początek_urlopu_2025 = urlopy[kto][1][0]
        koniec_urlopu_2025 = urlopy[kto][1][1]
        #print(początek_urlopu_2024, koniec_urlopu_2024, początek_urlopu_2025, koniec_urlopu_2025)
        
        początek_ostatniej_wycieczki = kiedy_jedzie[kto][0]
        koniec_ostatniej_wycieczki =  kiedy_jedzie[kto][1]
        #print(początek_ostatniej_wycieczki, koniec_ostatniej_wycieczki)
    
        if (początek_urlopu_2024 <= początek <= koniec_urlopu_2024) or (początek_urlopu_2024 <= koniec <= koniec_urlopu_2024) or (początek_urlopu_2025 <= początek <= koniec_urlopu_2025) or (początek_urlopu_2025 <= koniec <= koniec_urlopu_2025) or (początek_ostatniej_wycieczki <= początek <= koniec_ostatniej_wycieczki) or (początek_ostatniej_wycieczki <= koniec <= koniec_ostatniej_wycieczki):
            #print("NIE MOŻNA DODAĆ PRACOWNIKA ", kto, początek)
            dodawanie = False
        
        if dodawanie:
            kiedy_jedzie[kto] = (początek, koniec)
            wycieczka.loc[i, "id_pracownika"] = kto
            i += 1
        #print("\n")
    
    return(wycieczka)

In [26]:
koordynator_wycieczka = naprawa_bledow_pracownik_wycieczka(koordynatorzy_wycieczka, koordynatorzy, urlopy_koordynatorzy, kiedy_jedzie_koordynator)
fotograf_wycieczka = naprawa_bledow_pracownik_wycieczka(fotografowie_wycieczka, fotografowie, urlopy_fotografowie, kiedy_jedzie_fotograf)

In [27]:
print("Koordynatorzy", "\n")

wiersze = koordynatorzy_wycieczka.shape[0]
stosunki_wycieczek = []
stosunki_czasu = []

for i in koordynatorzy:
    print(i)
    stosunek1 = koordynatorzy_wycieczka[koordynatorzy_wycieczka["id_pracownika"]==i].shape[0]/koordynatorzy_wycieczka.shape[0]
    print("Stosunek wycieczek: ", stosunek1)
    stosunki_wycieczek.append(stosunek1)

    stosunek2 = sum(koordynatorzy_wycieczka[koordynatorzy_wycieczka["id_pracownika"]==i]["ile_trwa"])/sum(koordynatorzy_wycieczka["ile_trwa"])
    print("Stosunek czasu: ", stosunek2)
    stosunki_czasu.append(stosunek2)
    print("\n")

print(sum(stosunki_wycieczek))
print(sum(stosunki_czasu))

Koordynatorzy 

9
Stosunek wycieczek:  0.2522255192878338
Stosunek czasu:  0.25544554455445545


4
Stosunek wycieczek:  0.24925816023738873
Stosunek czasu:  0.22376237623762377


3
Stosunek wycieczek:  0.24925816023738873
Stosunek czasu:  0.24752475247524752


8
Stosunek wycieczek:  0.24925816023738873
Stosunek czasu:  0.2732673267326733


1.0
1.0


In [28]:
print("Fotografowie", "\n")

wiersze = fotograf_wycieczka.shape[0]
stosunki_wycieczek = []
stosunki_czasu = []

for i in fotografowie:
    print(i)
    stosunek1 = fotograf_wycieczka[fotograf_wycieczka["id_pracownika"]==i].shape[0]/fotograf_wycieczka.shape[0]
    print("Stosunek wycieczek: ", stosunek1)
    stosunki_wycieczek.append(stosunek1)

    stosunek2 = sum(fotograf_wycieczka[fotograf_wycieczka["id_pracownika"]==i]["ile_trwa"])/sum(fotograf_wycieczka["ile_trwa"])
    print("Stosunek czasu: ", stosunek2)
    stosunki_czasu.append(stosunek2)
    print("\n")

print(sum(stosunki_wycieczek))
print(sum(stosunki_czasu))

Fotografowie 

14
Stosunek wycieczek:  0.2522255192878338
Stosunek czasu:  0.2514851485148515


11
Stosunek wycieczek:  0.24629080118694363
Stosunek czasu:  0.2633663366336634


13
Stosunek wycieczek:  0.2433234421364985
Stosunek czasu:  0.24158415841584158


2
Stosunek wycieczek:  0.258160237388724
Stosunek czasu:  0.24356435643564356


1.0
1.0


In [29]:
#Łączymy i odpowiednio sortujemy DataFrames, a potem usuwamy niepotrzebne kolumny
pracownik_wycieczka = ((pd.concat([koordynator_wycieczka, fotograf_wycieczka], ignore_index=True)).drop(columns=['ile_trwa', 'data_rozpoczecia_wycieczki'])).sort_values(by='id_wycieczki')

## Generowanie urlopów dla pozostałych pracowników i tworzenie DataFrame z nimi

In [30]:
id_ochroniarz = stanowiska[stanowiska['stanowisko']=="Ochroniarz"]['id_stanowiska'].item()
id_sprzątaczka = stanowiska[stanowiska['stanowisko']=="Sprzątaczka"]['id_stanowiska'].item()
id_księgowa = stanowiska[stanowiska['stanowisko']=="Księgowa"]['id_stanowiska'].item()
id_sekretarka = stanowiska[stanowiska['stanowisko']=="Sekretarka"]['id_stanowiska'].item()
id_specjalista = stanowiska[stanowiska['stanowisko']=="Specjalista ds. Marketingu"]['id_stanowiska'].item()
id_organizator = stanowiska[stanowiska['stanowisko']=="Organizator wycieczki"]['id_stanowiska'].item()

daty_urlopu_ochroniarz = generator_urlopow(1); x1=0
daty_urlopu_sprzątaczka = generator_urlopow(1); x2=0
daty_urlopu_księgowa = generator_urlopow(1); x3=0
daty_urlopu_sekretarka = generator_urlopow(1); x4=0
daty_urlopu_specjalista = generator_urlopow(1); x5=0
daty_urlopu_organizatorów = generator_urlopow(2); x6=0

urlopy_pracowników = {}

for i in pracownicy["id_pracownika"]:
    if pracownicy.loc[i-1, "id_stanowiska"] == id_ochroniarz:
        urlopy_pracowników[i] = [daty_urlopu_ochroniarz[x1][0]]
        x1 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"]  == id_sprzątaczka:
        urlopy_pracowników[i] = [daty_urlopu_sprzątaczka[x2][0]]
        x2 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_księgowa:
        urlopy_pracowników[i] = [daty_urlopu_księgowa[x3][0]]
        x3 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_sekretarka:
        urlopy_pracowników[i] = [daty_urlopu_sekretarka[x4][0]]
        x4 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_specjalista:
        urlopy_pracowników[i] = [daty_urlopu_specjalista[x5][0]]
        x5 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_organizator:
        urlopy_pracowników[i] = [daty_urlopu_organizatorów[x6][0]]
        x6 +=1

for i in pracownicy["id_pracownika"]:
    if pracownicy.loc[i-1, "id_stanowiska"] == id_ochroniarz:
        urlopy_pracowników[i].append(daty_urlopu_ochroniarz[x1][0])
        x1 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"]  == id_sprzątaczka:
        urlopy_pracowników[i].append(daty_urlopu_sprzątaczka[x2][0])
        x2 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_księgowa:
        urlopy_pracowników[i].append(daty_urlopu_księgowa[x3][0])
        x3 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_sekretarka:
        urlopy_pracowników[i].append(daty_urlopu_sekretarka[x4][0])
        x4 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_specjalista:
        urlopy_pracowników[i].append(daty_urlopu_specjalista[x5][0])
        x5 +=1
    elif pracownicy.loc[i-1, "id_stanowiska"] == id_organizator:
        urlopy_pracowników[i].append(daty_urlopu_organizatorów[x6][0])
        x6 +=1

In [31]:
koordynator_urlop = {"id_pracownika": [], "data_rozpoczęcia": []}
for k in urlopy_koordynatorzy.keys():
    for i in range(2):
        koordynator_urlop["id_pracownika"].append(k)
        koordynator_urlop["data_rozpoczęcia"].append(urlopy_koordynatorzy[k][i][0])

fotograf_urlop = {"id_pracownika": [], "data_rozpoczęcia": []}
for k in urlopy_fotografowie.keys():
    for i in range(2):
        fotograf_urlop["id_pracownika"].append(k)
        fotograf_urlop["data_rozpoczęcia"].append(urlopy_fotografowie[k][i][0])

koordynator_urlop = pd.DataFrame(koordynator_urlop)
fotograf_urlop = pd.DataFrame(fotograf_urlop)

In [32]:
pracownik_urlopy = {"id_pracownika": [], "data_rozpoczęcia": []}

for klucz in urlopy_pracowników.keys():
    for wartość in urlopy_pracowników[klucz]:
        pracownik_urlopy["id_pracownika"].append(klucz)
        pracownik_urlopy["data_rozpoczęcia"].append(wartość)

pracownik_urlopy = pd.DataFrame(pracownik_urlopy)

In [33]:
pracownik_urlop = (pd.concat([pracownik_urlopy, koordynator_urlop, fotograf_urlop], axis=0, ignore_index=False)).sort_values(by="data_rozpoczęcia")

## Dostoswanie DataFrames, aby zapisane dane były zgodne ze schematem

In [34]:
osoby.drop(['zajeta', 'min_data'], axis=1, inplace=True)
osoby.reset_index(drop=True, inplace=True)

pracownicy = pracownicy[['id_pracownika', 'imie', 'nazwisko', 'id_stanowiska', 'data_zatrudnienia', 'czy_zatrudniony', 'numer_telefonu', 'e-mail']]
pracownicy.rename(columns={'e-mail': 'email'}, inplace=True)
pracownicy.reset_index(drop=True, inplace=True)

dane_alarmowe.drop('id_alarm_pomocnicze', axis=1, inplace=True)
dane_alarmowe = dane_alarmowe[['id_alarm', 'imie', 'nazwisko', 'numer_telefonu']]

jaka_wycieczka.drop(['długość', 'liczba_osób'], axis=1, inplace=True)
jaka_wycieczka = jaka_wycieczka[['id', 'id_planu', 'data']]
jaka_wycieczka.rename(columns={'id': 'id_wycieczki', 'id_planu': 'id_oferty', 'data': 'data_wycieczki'}, inplace=True)
jaka_wycieczka.reset_index(drop=True, inplace=True)

osoba_wycieczka.drop(['czy_zapłacono'], axis=1, inplace=True)
osoba_wycieczka.rename(columns={'data_zapłaty': 'data_platnosci'}, inplace=True)
osoba_wycieczka.reset_index(drop=True, inplace=True)

pracownik_wycieczka.reset_index(drop=True, inplace=True)

pracownik_urlop['koniec_urlopu'] = pracownik_urlop['data_rozpoczęcia'] + timedelta(days=19)
pracownik_urlop.rename(columns={'data_rozpoczęcia': 'poczatek_urlopu'}, inplace=True)
pracownik_urlop.reset_index(drop=True, inplace=True)

oferty.rename(columns={'id': 'id_oferty'}, inplace=True)
oferty['id_cennika'] = oferty['id_oferty']
oferty = oferty[['id_oferty', 'nazwa', 'id_rodzaju', 'id_kierunku', 'id_miasta', 'opis', 'id_cennika', 'czas_trwania', 'czy_bezpieczna']]

df_rodzaje.rename(columns={'rodzaj': 'nazwa'}, inplace=True)

df_miasta.rename(columns={'miasto': 'nazwa'}, inplace=True)

cennik.rename(columns={'koszt_organizacji': 'koszt_podstawowy'}, inplace=True)

## Zapisywanie plików

In [35]:
osoby.to_csv('../Dane do tabel/osoby.csv', index=False)

pracownicy.to_csv('../Dane do tabel/pracownicy.csv', index=False)

dane_alarmowe.to_csv('../Dane do tabel/dane_alarmowe.csv', index=False)

jaka_wycieczka.to_csv('../Dane do tabel/zorganizowana_wycieczka.csv', index=False)

osoba_wycieczka.to_csv('../Dane do tabel/osoby_zorganizowana_wycieczka.csv', index=False)

pracownik_wycieczka.to_csv('../Dane do tabel/pracownicy_wycieczka.csv', index=False)

pracownik_urlop.to_csv('../Dane do tabel/pracownik_urlop.csv', index=False)

oferty.to_csv('../Dane do tabel/oferty.csv', index=False)

df_rodzaje.to_csv('../Dane do tabel/rodzaje.csv', index = False)

df_miasta.to_csv('../Dane do tabel/miasta.csv', index = False)

df_kier.to_csv('../Dane do tabel/kierunki.csv', index = False)

cennik.to_csv('../Dane do tabel/cennik.csv', index = False)

stanowiska.to_csv('../Dane do tabel/stanowiska.csv', index = False)